# Examples for QMSim.jl

This demonstrates the basic functionality of `QMSim.jl` from matrix construction all the way to solving real quantum mechanics problems.

In [1]:
# activate the examples environment
import Pkg;
Pkg.activate(@__DIR__)
# Pkg.add("Term")
# Pkg.resolve()
# Pkg.instantiate()
# Pkg.precompile()
# Pkg.update()

  Activating project at `~/Julia/QMSim.jl/examples`


In [2]:
ENV["COLUMNS"] = 160

using Revise
using SparseArrays, LinearAlgebra

using QGas.NumericalTools.ArrayDimensions: Dimensions

using QMSim
using QMSim.Solvers

import QMSim: error_check

## Demonstrate the creation of some simple rules

In [3]:
    function ham_tunneling(x, y; J=1.0)
        return -J
    end

    RelativeRule(ham_tunneling, [1,])
    ExplicitRule( Diagonal(collect(1:5) .+ 0.0im) )

ExplicitRule{Diagonal{ComplexF64, Vector{ComplexF64}}}(ComplexF64[1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 2.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 4.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 5.0 + 0.0im])

## Demonstrate the overall setup of a matrix

This does not solve any problems.

In [4]:
# create an array of physical dimensions for our quantum system to live in, including an example with two space and one spin dimensions

dims = Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=5, unit="X Momentum", periodic=true, spatial=true),
    # DimensionWithSpace(; x0=-1.0, dx=1.0, npnts=3, unit="Spin", periodic=true, spatial=true),
)

"""
    build_rules!(mwr::MatrixWithRules)

add required rules to the matrix
"""
function build_rules!(mwr::MatrixWithRules)

    function ham_tunneling(x, y; J=1.0)
        return -J
    end

    add_rule!(mwr, RelativeRule, ham_tunneling, [1,])
    add_rule!(mwr, RelativeRule, ham_tunneling, [-1,])
    add_rule!(mwr, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))
    return mwr
end

# See how we define the type of the matrix we want!
mwr = MatrixWithRules(SparseMatrixCSC{ComplexF64}, dims)

build_rules!(mwr)

generate_builders!(mwr)

build!(mwr; J=2.0)

mwr

5×5 MatrixWithRules{ComplexF64, SparseMatrixCSC{ComplexF64}}:
  1.0+0.0im  -2.0+0.0im   0.0+0.0im   0.0+0.0im  -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im  -2.0+0.0im   3.0+0.0im  -2.0+0.0im   0.0+0.0im
  0.0+0.0im   0.0+0.0im  -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im   0.0+0.0im   0.0+0.0im  -2.0+0.0im   5.0+0.0im

Now we do the same thing, but for more than one matrix:

In [5]:
mwrs = MatricesWithRules(SparseMatrixCSC{ComplexF64}, dims)

add_leaf!(mwrs, :tunneling)
add_leaf!(mwrs, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(mwrs, :potential, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))

generate_builders!(mwrs)

build!(mwrs; J=2.0)

mwrs.matrix

5×5 SparseMatrixCSC{ComplexF64, Int64} with 15 stored entries:
  1.0+0.0im  -2.0+0.0im       ⋅           ⋅      -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im       ⋅           ⋅    
      ⋅      -2.0+0.0im   3.0+0.0im  -2.0+0.0im       ⋅    
      ⋅           ⋅      -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im       ⋅           ⋅      -2.0+0.0im   5.0+0.0im

In [6]:
mwrs = MatricesWithRules(SparseMatrixCSC{ComplexF64}, dims)

add_leaf!(mwrs, :tunneling)
add_leaf!(mwrs, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(mwrs, :potential, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))

set_default_kwargs!(mwrs, :tunneling; J=4.0)

generate_builders!(mwrs)

build!(mwrs)

mwrs.matrix

5×5 SparseMatrixCSC{ComplexF64, Int64} with 15 stored entries:
  1.0+0.0im  -4.0+0.0im       ⋅           ⋅      -4.0+0.0im
 -4.0+0.0im   2.0+0.0im  -4.0+0.0im       ⋅           ⋅    
      ⋅      -4.0+0.0im   3.0+0.0im  -4.0+0.0im       ⋅    
      ⋅           ⋅      -4.0+0.0im   4.0+0.0im  -4.0+0.0im
 -4.0+0.0im       ⋅           ⋅      -4.0+0.0im   5.0+0.0im

At this point this is not all that useful, but the main point is that with slightly more syntactic sugar we can write code to define pretty much any physics problem!

## Solve specific problems

The existing code already encapsulates any abstract matrix.  There is some work required to make this work efficiently for both dense and sparse matrices.

### ERD SOC

I am going to start with the simple example of ERD Spin orbit coupling to show the behavior of a completed module.

In [20]:
# Start with the dimensions for F = 1
dims = Dimensions(
    DimensionWithSpace(; x0=-1.0, dx=1.0, npnts=3, unit="Spin", periodic=false, spatial=false),
)

solver = ERD_SOC_Solver(dims)

build!(solver)

eigensystem!(solver.mwrs; q=0.1, Ω=1.0im)

solver.mwrs.eigenvalues

3-element Vector{ComplexF64}:
 -0.6148924330173668 + 0.0im
  0.9728183395439645 + 0.0im
  1.6720740934734044 + 0.0im

### Other examples

Here is a more general example for a dense matrix including performance profiling

In [8]:
dims = Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=401, unit="X Momentum", periodic=true, spatial=true),
)

qms = QMSolver(Matrix{ComplexF64}, dims; num_states=6, wrap=Hermitian)

add_leaf!(qms, :tunneling)
add_leaf!(qms, :potential)
add_rule!(qms, :potential, ExplicitRule, Diagonal(4 .* ones(ComplexF64, 401)))

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(qms, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(qms, :tunneling, RelativeRule, ham_tunneling, [-1,])

generate_builders!(qms)

eigensystem!(qms; J=2.0)
qms.eigenvalues

401-element Vector{ComplexF64}:
 -2.1884719226525088e-16 + 0.0im
   0.0004910119951010936 + 0.0im
   0.0004910119951014893 + 0.0im
   0.0019639274340130723 + 0.0im
    0.001963927434013853 + 0.0im
    0.004418384707162504 + 0.0im
    0.004418384707162711 + 0.0im
    0.007853781230567063 + 0.0im
    0.007853781230567142 + 0.0im
    0.012269273593776529 + 0.0im
                         ⋮
       7.990060922452623 + 0.0im
       7.993986487366496 + 0.0im
       7.993986487366498 + 0.0im
      7.9969315046435865 + 0.0im
      7.9969315046435865 + 0.0im
      7.9988952512644875 + 0.0im
       7.998895251264488 + 0.0im
       7.999877245117628 + 0.0im
        7.99987724511763 + 0.0im

In [9]:
"""
    NpodSolver <: AbstractMatrixSolver

A subtype of `AbstractMatrixSolver` that setups and solves the N-pod problem.  This describes two-level atom, a lambda-scheme, a tripod, and anything with one level coupled to N-sub-levels. 
"""
mutable struct NpodSolver{T, M<:AbstractMatrix{T}} <: AbstractMatrixSolver{T,M}
    shared::MatrixSharedData
    mwrs::QMSolver{T, M}
    N::Int # number of sub-levels (caps by convention here)
end
issolver(::Type{<:NpodSolver}) = SolverTrait()

function NpodSolver(adims::Dimensions)

    shared = MatrixSharedData(adims)    

    mwrs = QMSolver(Matrix{ComplexF64}, shared)

    NpodSolver(shared, mwrs, dim(mwrs))
end

"""
    error_checks(solver::NpodSolver)

This will check that the N-pod solver is setup correctly.
"""
function error_check(solver::NpodSolver)
    if ndims(get_dimensions(solver)) != 1
        throw(ArgumentError("N-pod solver must have a single dimension"))
    end

    error_check(get_leaf(solver))
end

"""
    build_rules!(nps::NpodSolver)

This will configure all the rules for the N-pod solver.  We assume that the first
state in the basis is the state which is coupled to the remaining levels.
"""
function build_rules!(solver::NpodSolver)

    add_leaf!(solver, :hamiltonian)

    function coupling(x0, x1; Ωs=[1.0, 1.0, 1.0, 1.0, 1.0])
        Δ = Int(x0[1] - x1[1])
        Ω = Δ > 0 ? Ωs[Δ] : conj(Ωs[-Δ])
        return Ω
    end

    i = 1
    for j in 2:solver.N
        add_rule!(solver, :hamiltonian, ElementRule, coupling, [i,], [j,])
        add_rule!(solver, :hamiltonian, ElementRule, coupling, [j,], [i,])
    end

    add_rule!(solver, :hamiltonian, ExplicitRule, Diagonal(4 .* ones(ComplexF64, 5)))


    set_default_kwargs!(solver, :hamiltonian) # TODO: need to implement this for links.
end

#
# Test it out!
#

dims = Dimensions(
    DimensionWithSpace(; x0=0, dx=1.0, npnts=5, unit="Internal State", periodic=false, spatial=false),
)

solver = NpodSolver(dims)
error_check(solver)

build_rules!(solver) # Todo generate_builders!(qms) vs build rules

generate_builders!(solver)

eigensystem!(solver; Ωs=[1.0, 2.0, 3.0, 4.0, 5.0])


MethodError: MethodError: no method matching eigensystem!(::NpodSolver{ComplexF64, Matrix{ComplexF64}}; Ωs::Vector{Float64})
The function `eigensystem!` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  eigensystem!(!Matched::QMSolver; names, which, kwargs...)
   @ QMSim ~/Julia/QMSim.jl/src/MatrixSolvers.jl:109
